In [28]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql

In [68]:
ip_address = os.environ['AWS_IP']
PASSWORD = os.environ['SQL_PWD_AWS']

con = pymysql.connect(
    user='edward', 
    passwd=PASSWORD,
    host=ip_address, 
    db='community', 
    charset='utf8'
)

In [ ]:
BASE_URL = 'https://pann.nate.com'

In [34]:
def continuos_dates(start, end, fm):
    dates = []
    dt = datetime.datetime.strptime(start, fm)
    while True:
        dt_string = dt.strftime(format=fm)
        dates.append(dt_string)
        if dt_string == end:
            break
        dt = dt + datetime.timedelta(days=1)
    return dates

In [72]:
def get_urls(date):
    urls = []
    for page in range(1, 3):
        params = {'stdt': date, 'page': page}
        result = requests.get(BASE_URL + '/talk/ranking/d', params = params)
        soup = BeautifulSoup(result.text, 'lxml')
        lists = soup.find('ul', {'class':'post_wrap'}).find_all('li')
        urls.extend([['nate_pann', li.find('dt').find('a')['href'], date] for li in lists])
    return urls

In [70]:
def commit_sql(data):
    with  con.cursor() as cursor:
        cursor.executemany("insert into urls(name, url, date) values (%s, %s, %s)", data )
        con.commit()

In [ ]:
start = '20210101'
end = '20210811'
dates = continuos_dates(start, end, '%Y%m%d')

In [58]:
for date in dates:
    urls = get_urls(date)
    commit_sql(urls)